<a href="https://colab.research.google.com/github/kiet-na/mtcnn_facenet/blob/main/FaceNet_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Tech learning

/content/drive/MyDrive/Tech learning


In [ ]:
pwd

'/content/drive/MyDrive/Tech learning'

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [ ]:
data_dir = '/content/drive/MyDrive/Tech learning/test/dataset'

batch_size = 32
epochs = 8
workers = 8 if os.name == 'nt' else 8

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

/content/drive/MyDrive/Tech learning/facenet_pytorch/models/inception_resnet_v1.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)


In [ ]:
!pip install torchinfo

In [ ]:
# Print with torchinfo
from torchinfo import summary

summary(model = resnet,
        input_size = (1,3,512,512),
        col_names = ["input_size","output_size","num_params","trainable"],
        col_width = 20,
        row_settings = ["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
InceptionResnetV1 (InceptionResnetV1)         [1, 3, 512, 512]     [1, 5]               --                   True
├─BasicConv2d (conv2d_1a)                     [1, 3, 512, 512]     [1, 32, 255, 255]    --                   True
│    └─Conv2d (conv)                          [1, 3, 512, 512]     [1, 32, 255, 255]    864                  True
│    └─BatchNorm2d (bn)                       [1, 32, 255, 255]    [1, 32, 255, 255]    64                   True
│    └─ReLU (relu)                            [1, 32, 255, 255]    [1, 32, 255, 255]    --                   --
├─BasicConv2d (conv2d_2a)                     [1, 32, 255, 255]    [1, 32, 253, 253]    --                   True
│    └─Conv2d (conv)                          [1, 32, 255, 255]    [1, 32, 253, 253]    9,216                True
│    └─BatchNorm2d (bn)                       [1, 32, 253, 253]    [1, 32, 253, 253] 

In [ ]:
for param in resnet.conv2d_1a.parameters():
    param.requires_grad = False
for param in resnet.conv2d_2a.parameters():
    param.requires_grad = False
for param in resnet.conv2d_2b.parameters():
    param.requires_grad = False
for param in resnet.conv2d_3b.parameters():
    param.requires_grad = False
for param in resnet.conv2d_4a.parameters():
    param.requires_grad = False
for param in resnet.conv2d_4b.parameters():
    param.requires_grad = False
for param in resnet.repeat_1.parameters():
    param.requires_grad = False
for param in resnet.repeat_2.parameters():
    param.requires_grad = False
for param in resnet.repeat_3.parameters():
    param.requires_grad = False
for param in resnet.mixed_6a.parameters():
    param.requires_grad = False
for param in resnet.mixed_7a.parameters():
    param.requires_grad = False
for param in resnet.block8.parameters():
    param.requires_grad = False

In [ ]:
# Print with torchinfo
from torchinfo import summary

summary(model = resnet,
        input_size = (1,3,512,512),
        col_names = ["input_size","output_size","num_params","trainable"],
        col_width = 20,
        row_settings = ["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
InceptionResnetV1 (InceptionResnetV1)         [1, 3, 512, 512]     [1, 5]               --                   Partial
├─BasicConv2d (conv2d_1a)                     [1, 3, 512, 512]     [1, 32, 255, 255]    --                   False
│    └─Conv2d (conv)                          [1, 3, 512, 512]     [1, 32, 255, 255]    (864)                False
│    └─BatchNorm2d (bn)                       [1, 32, 255, 255]    [1, 32, 255, 255]    (64)                 False
│    └─ReLU (relu)                            [1, 32, 255, 255]    [1, 32, 255, 255]    --                   --
├─BasicConv2d (conv2d_2a)                     [1, 32, 255, 255]    [1, 32, 253, 253]    --                   False
│    └─Conv2d (conv)                          [1, 32, 255, 255]    [1, 32, 253, 253]    (9,216)              False
│    └─BatchNorm2d (bn)                       [1, 32, 253, 253]    [1, 32, 25

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    transforms.Resize((512, 512)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder(data_dir, transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
val_inds

array([16, 11, 12, 29, 19, 23, 17])

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    1.5518 | fps:    0.2249 | acc:    0.1429   

Epoch 1/8
----------
Train |     1/1    | loss:    1.9085 | fps:    0.5250 | acc:    0.0741   
Valid |     1/1    | loss:    1.3574 | fps:    0.7285 | acc:    0.4286   

Epoch 2/8
----------
Train |     1/1    | loss:    0.6029 | fps:    0.5351 | acc:    0.9630   
Valid |     1/1    | loss:    1.2280 | fps:    0.8511 | acc:    0.5714   

Epoch 3/8
----------
Train |     1/1    | loss:    0.2340 | fps:    0.5533 | acc:    0.9630   
Valid |     1/1    | loss:    1.1128 | fps:    0.5973 | acc:    0.5714   

Epoch 4/8
----------
Train |     1/1    | loss:    0.1066 | fps:    0.5673 | acc:    1.0000   
Valid |     1/1    | loss:    1.0355 | fps:    0.7442 | acc:    0.5714   

Epoch 5/8
----------
Train |     1/1    | loss:    0.0501 | fps:    0.5129 | acc:    1.0000   
Valid |     1/1    | loss:    0.9793 | fps:    0.8552 | acc:    0.5714   

Epoch 6/8
----------
Train |     1/1    | loss:    0.0

#No freeze

In [ ]:
for param in resnet.conv2d_1a.parameters():
    param.requires_grad = True
for param in resnet.conv2d_2a.parameters():
    param.requires_grad = True
for param in resnet.conv2d_2b.parameters():
    param.requires_grad = True
for param in resnet.conv2d_3b.parameters():
    param.requires_grad = True
for param in resnet.conv2d_4a.parameters():
    param.requires_grad = True
for param in resnet.conv2d_4b.parameters():
    param.requires_grad = True
for param in resnet.repeat_1.parameters():
    param.requires_grad = True
for param in resnet.repeat_2.parameters():
    param.requires_grad = True
for param in resnet.repeat_3.parameters():
    param.requires_grad = True
for param in resnet.mixed_6a.parameters():
    param.requires_grad = True
for param in resnet.mixed_7a.parameters():
    param.requires_grad = True
for param in resnet.block8.parameters():
    param.requires_grad = True

NameError: name 'resnet' is not defined

In [ ]:
# Print with torchinfo
from torchinfo import summary

summary(model = resnet,
        input_size = (1,3,512,512),
        col_names = ["input_size","output_size","num_params","trainable"],
        col_width = 20,
        row_settings = ["var_names"])

NameError: name 'resnet' is not defined

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    transforms.Resize((512, 512)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder(data_dir, transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()

# inference

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

/content/drive/MyDrive/Tech learning/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
/content/driv

In [ ]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('/content/drive/MyDrive/Tech learning/test/infer')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)
print(dataset.class_to_idx)


{'Kiet': 0, 'Nhan': 1}


In [ ]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999979
Face detected with probability: 0.999965
Face detected with probability: 0.999470
Face detected with probability: 0.999991


In [ ]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [ ]:
import pandas as pd

In [ ]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

          Kiet      Kiet      Nhan      Nhan
Kiet  0.000000  2.657559  7.915783  6.754677
Kiet  2.657559  0.000000  6.020245  5.048040
Nhan  7.915783  6.020245  0.000000  2.015617
Nhan  6.754677  5.048040  2.015617  0.000000
